# Getting Started

In this tutorial, we will:
* install a local PostgreSQL database,
* load a few tables,
* install `pyqrlew` (Qrlew python binding),
* connect to the database and create a `pyqrlew.Dataset`
* set bounds to some variables
* write a query and rewrite it as differentially private